<a href="https://colab.research.google.com/github/Angelawork/AI4Good_LiteraLingo_M3Project/blob/main/Datasets/Metaphors_T5_fine_tuning_Chantal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T5 model tested on metaphors. 
Chantal

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from google.colab import files
uploaded = files.upload()

Saving Metaphor.csv to Metaphor.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Metaphor.csv']))
# Dataset is now stored in a Pandas Dataframe
print(df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("figurative-nlp/t5-figurative-paraphrase")


In [ ]:
import pandas as pd
from transformers import AutoTokenizer


# Initialize the tokenizer using the same tokenizer as your pre-trained model
tokenizer = AutoTokenizer.from_pretrained("figurative-nlp/t5-figurative-paraphrase")

# Create empty lists to store the tokenized inputs and outputs
tokenized_inputs = []
tokenized_outputs = []

# Iterate over the rows of the DataFrame and tokenize the input and output columns
for index, row in df.iterrows():
    input_text = row["Figurative sentence"]
    output_text = row["Literal meaning"]
    
    tokenized_input = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    tokenized_output = tokenizer(output_text, return_tensors="pt", truncation=True, padding=True)
    
    tokenized_inputs.append(tokenized_input)
    tokenized_outputs.append(tokenized_output)


In [ ]:
import torch
from torch.utils.data import Dataset

class FigurativeDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

additional_data = [
    ("She has a heart of gold.", "She is very kind and generous."),
    ("He is a couch potato.", "He is lazy and spends a lot of time sitting and watching TV."),
    ("Life is a rollercoaster.", "Life has its ups and downs."),
    # Add more examples...
]

# Tokenize the additional data
additional_tokenized_data = []
for fig_expr, lit_expr in additional_data:
    tokenized_input = tokenizer.encode_plus(fig_expr, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(lit_expr, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
    
    additional_tokenized_data.append({
        "input_ids": tokenized_input["input_ids"].squeeze(),
        "attention_mask": tokenized_input["attention_mask"].squeeze(),
        "labels": tokenized_output["input_ids"].squeeze(),
    })

# Create the initial dataset
tokenized_data = additional_tokenized_data

# Create an instance of the dataset
dataset = FigurativeDataset(tokenized_data)


In [ ]:
from torch.utils.data import random_split

# Define the percentage of the dataset to use for validation
validation_percentage = 0.5

# Calculate the number of samples for validation
num_validation_samples = int(validation_percentage * len(dataset))

# Calculate the number of samples for training
num_training_samples = len(dataset) - num_validation_samples

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(dataset, [num_training_samples, num_validation_samples])


In [ ]:
print(num_validation_samples)

1


In [ ]:
from torch.utils.data import DataLoader

# Define batch size for training and validation
batch_size = 8

# Create data loaders for training and validation
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AdamW

# Initialize the model
model = AutoModelForSeq2SeqLM.from_pretrained("figurative-nlp/t5-figurative-paraphrase")

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-3)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Set the model in training mode
model.train()

# Define the number of training epochs
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    
    for batch in train_data_loader:
        # Move the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(**batch)
        
        # Compute the loss
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Update the model parameters
        optimizer.step()
        
        total_loss += loss.item()
    
    # Calculate average training loss for the epoch
    avg_loss = total_loss / len(train_data_loader)
    
    # Print training loss for the epoch
    print(f"Epoch {epoch + 1} - Training loss: {avg_loss:.4f}")


Epoch 1 - Training loss: 21.2102
Epoch 2 - Training loss: 13.1187
Epoch 3 - Training loss: 1.7518
Epoch 4 - Training loss: 1.0302
Epoch 5 - Training loss: 0.5959


In [ ]:
# Set the model in evaluation mode
model.eval()

total_loss = 0
num_batches = len(val_data_loader)

# Check if the validation data loader is not empty
if num_batches > 0:
    # Disable gradient calculation
    with torch.no_grad():
        for batch in val_data_loader:
            # Move the batch to the device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward pass
            outputs = model(**batch)

            # Compute the loss
            loss = outputs.loss

            total_loss += loss.item()

        # Calculate average validation loss
        avg_loss = total_loss / num_batches

        # Print validation loss
        print(f"Validation loss: {avg_loss:.4f}")
else:
    print("Validation data loader is empty.")


Validation loss: 0.2461


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("figurative-nlp/t5-figurative-paraphrase")
model = AutoModelForSeq2SeqLM.from_pretrained("figurative-nlp/t5-figurative-paraphrase")

input_text = "The moon winked back at itself from the lake surface."

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate outputs
outputs = model.generate(input_ids=input_ids, num_beams=5)

# Decode the generated outputs
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(output_text)


The moon looked back at itself from the lake surface.
